In [ ]:
import json
from pandas import DataFrame
from pymatgen import MPRester, Structure
from mpcontribs.client import load_client

In [ ]:
with open('/Users/patrick/Downloads/log_kvrh', 'r') as f:
    data = json.load(f)
df = DataFrame(data['data'], columns=data['columns'])

In [ ]:
project = 'matbench_v1'
client = load_client('your-api-key-here')
mpr = MPRester()

Get project info:

In [ ]:
client.projects.get_entry(pk=project).result()

Create contributions with data and add structures:

In [ ]:
for idx, (s, kvrh) in enumerate(zip(df['structure'], df['log10(K_VRH)'])):
    structure = Structure.from_dict(s)
    matches = mpr.find_structure(structure)
    if not matches:
        print('no match for idx', idx)
        continue
    identifier = matches[0]
    # dupe check
    contrib = client.contributions.get_entries(
        project=project, identifier=identifier, _limit=1
    ).result()
    if contrib['total_count'] > 0:
        print(identifier, 'already added:', contrib['data'][0]['id'])
        continue
    # create contribution
    contrib = {'project': project, 'identifier': identifier, 'data': {'K|VRH': kvrh}}
    cid = client.contributions.create_entry(contribution=contrib).result()['id']
    # add structure to contribution
    sdct = dict(contribution=cid, name=structure.formula.replace(' ', ''), label='2020/02/02')
    sdct.update(structure.as_dict())
    try:
        sid = client.structures.create_entry(structure=sdct).result()['id']
    except Exception as ex:
        print(ex)
        print(sdct)
        client.contributions.delete_entry(pk=cid).result()
        break
    print(idx, cid)
    if idx > 5:
        break

Retrieve and download contributions:

In [ ]:
contribs = client.contributions.get_entries(project=project, _fields=['id'], _limit=100).result()
contribs['total_count'], len(contribs['data'])

Delete entries:

In [ ]:
for idx, contrib in enumerate(contribs['data']):
    client.contributions.delete_entry(pk=contrib['id']).result()
    print(idx)